In [1]:
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import seaborn as sns

In [9]:
# from sklearn.naive_bayes import MultinomialNB - хуже работает
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [2]:
%%time
import pickle
with open('page_dict.p', 'rb') as fp:
    page_dict = pickle.load(fp)

CPU times: user 5.55 s, sys: 2.66 s, total: 8.21 s
Wall time: 10.7 s


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
train_data = pd.read_csv('./anomaly-detection-competition-ml1-ts-spring-2020/train_groups.csv')
test_data = pd.read_csv('./anomaly-detection-competition-ml1-ts-spring-2020/test_groups.csv')

In [5]:
train_documents = []
for i in train_data['doc_id']:
    train_documents.append(page_dict.get(i))

In [6]:
vectorizer = CountVectorizer(max_features=500)
vectors = vectorizer.fit_transform(train_documents)

In [7]:
y = np.array(train_data['target'])

In [10]:
clf = GaussianNB(var_smoothing=1e-12)

In [11]:
cross_val_score(clf, vectors.toarray(), y, cv=5, scoring='f1').mean()

0.5691284198009627

In [12]:
test_documents = []
for i in test_data['doc_id']:
    test_documents.append(page_dict.get(i))

In [13]:
vectors_train = vectorizer.fit_transform(test_documents)

In [14]:
clf = GaussianNB(var_smoothing=1e-12)

In [15]:
clf.fit(vectors.toarray(), y)

GaussianNB(priors=None, var_smoothing=1e-12)

In [16]:
test_data['target']  =  clf.predict(vectors_train.toarray())

In [17]:
test_data[['pair_id', 'target']].to_csv('try_naive.csv', index=None)